In [1]:
import pandas as pd
import numpy as np
import os

# Zip Code Level Data

In [2]:
df_farms_1 = pd.read_csv("Farms 1.csv")
df_farms_1

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2017,YEAR,NaN,ZIP CODE,ALABAMA,1,NaN,NaN,NaN,...,35004,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),6,NaN
1,CENSUS,2017,YEAR,NaN,ZIP CODE,ALABAMA,1,NaN,NaN,NaN,...,35005,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),5,NaN
2,CENSUS,2017,YEAR,NaN,ZIP CODE,ALABAMA,1,NaN,NaN,NaN,...,35006,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),27,NaN
3,CENSUS,2017,YEAR,NaN,ZIP CODE,ALABAMA,1,NaN,NaN,NaN,...,35006,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),2,NaN
4,CENSUS,2017,YEAR,NaN,ZIP CODE,ALABAMA,1,NaN,NaN,NaN,...,35007,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48350,CENSUS,2017,YEAR,NaN,ZIP CODE,NORTH DAKOTA,38,NaN,NaN,NaN,...,58856,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),20,NaN
48351,CENSUS,2017,YEAR,NaN,ZIP CODE,NORTH DAKOTA,38,NaN,NaN,NaN,...,58856,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (TENANT),2,NaN
48352,CENSUS,2017,YEAR,NaN,ZIP CODE,NORTH DAKOTA,38,NaN,NaN,NaN,...,99999,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),"2,234",NaN
48353,CENSUS,2017,YEAR,NaN,ZIP CODE,NORTH DAKOTA,38,NaN,NaN,NaN,...,99999,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),280,NaN


In [3]:
df_farms_1["Zip Code"] = df_farms_1["Zip Code"].astype(str).str.zfill(5)
df_farms_1['Value'] = pd.to_numeric(df_farms_1['Value'].str.replace(',', ''), errors='coerce')
df_farms_1.rename(columns={'Value': 'Num Farms'}, inplace=True)
grouped_df = df_farms_1.groupby('Zip Code')['Num Farms'].sum().reset_index()
grouped_df.set_index('Zip Code', inplace=True)
grouped_df

,Num Farms
Zip Code,
01001,11
01002,60
01003,3
01004,3
01005,60
...,...
99926,1
99928,1
99929,2


In [4]:
df_farms_2 = pd.read_csv("Farms 2.csv")
df_farms_2["Zip Code"] = df_farms_2["Zip Code"].astype(str).str.zfill(5)
df_farms_2['Value'] = pd.to_numeric(df_farms_2['Value'].str.replace(',', ''), errors='coerce')
df_farms_2.rename(columns={'Value': 'Num Farms'}, inplace=True)
df_farms_2

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Num Farms,CV (%)
0,CENSUS,2017,YEAR,NaN,ZIP CODE,OHIO,39,NaN,NaN,NaN,...,43001,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),69,NaN
1,CENSUS,2017,YEAR,NaN,ZIP CODE,OHIO,39,NaN,NaN,NaN,...,43001,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),15,NaN
2,CENSUS,2017,YEAR,NaN,ZIP CODE,OHIO,39,NaN,NaN,NaN,...,43001,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (TENANT),6,NaN
3,CENSUS,2017,YEAR,NaN,ZIP CODE,OHIO,39,NaN,NaN,NaN,...,43002,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),2,NaN
4,CENSUS,2017,YEAR,NaN,ZIP CODE,OHIO,39,NaN,NaN,NaN,...,43003,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26001,CENSUS,2017,YEAR,NaN,ZIP CODE,WYOMING,56,NaN,NaN,NaN,...,83414,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),9,NaN
26002,CENSUS,2017,YEAR,NaN,ZIP CODE,WYOMING,56,NaN,NaN,NaN,...,83414,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (TENANT),6,NaN
26003,CENSUS,2017,YEAR,NaN,ZIP CODE,WYOMING,56,NaN,NaN,NaN,...,99999,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (FULL OWNER),542,NaN
26004,CENSUS,2017,YEAR,NaN,ZIP CODE,WYOMING,56,NaN,NaN,NaN,...,99999,NaN,0,NaN,FARM OPERATIONS,FARM OPERATIONS - NUMBER OF OPERATIONS,TENURE,TENURE: (PART OWNER),111,NaN


In [5]:
grouped_df_2 = df_farms_2.groupby('Zip Code')['Num Farms'].sum().reset_index()
grouped_df_2.set_index('Zip Code', inplace=True)
grouped_df_2

,Num Farms
Zip Code,
02804,22
02806,2
02807,6
02808,9
02809,18
...,...
99371,20
99401,23
99402,56


In [6]:
df_farms = grouped_df.add(grouped_df_2, fill_value=0)
df_farms

,Num Farms
Zip Code,
01001,11.0
01002,60.0
01003,3.0
01004,3.0
01005,60.0
...,...
99926,1.0
99928,1.0
99929,2.0


The zip code 99999 represents areas which are not included in any zip code (military bases, federal lands, etc.). This line will be removed prior to statistical analysis but is computationally convenient to leave in the data until the end due to its presence in other data files.

In [7]:
cbp_df = pd.read_csv("zbp21totals.csv", encoding='latin1')
cbp_df["zip"] = cbp_df["zip"].astype(str).str.zfill(5)
cbp_df.set_index("zip", inplace=True)
cbp_df

,name,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,city,stabbr,cty_name
zip,,,,,,,,,,,
00501,"HOLTSVILLE, NY",J,41,J,198,J,898,10,HOLTSVILLE,NY,SUFFOLK
01001,"AGAWAM, MA",G,7616,G,103920,G,447397,475,AGAWAM,MA,HAMPDEN
01002,"AMHERST, MA",H,7450,H,70191,H,315640,502,AMHERST,MA,HAMPSHIRE
01003,"AMHERST, MA",G,214,H,2166,H,9307,15,AMHERST,MA,HAMPSHIRE
01004,"AMHERST, MA",J,117,J,1162,J,5618,6,AMHERST,MA,HAMPSHIRE
...,...,...,...,...,...,...,...,...,...,...,...
99926,"METLAKATLA, AK",J,57,H,486,H,2131,9,METLAKATLA,AK,PRINCE OF WALES-HYDER CENSUS A
99928,"WARD COVE, AK",G,13,H,101,H,619,4,WARD COVE,AK,KETCHIKAN GATEWAY BOROUGH
99929,"WRANGELL, AK",H,585,H,6434,H,23758,99,WRANGELL,AK,WRANGELL CITY AND BOROUGH


In [8]:
master_df = pd.concat([df_farms, cbp_df["est"]], axis=1)
master_df = master_df.fillna(0).sort_index()
master_df.rename(columns={'est': 'Num Estabs'}, inplace=True)
master_df["Total Emp Bus"] = master_df["Num Farms"] + master_df["Num Estabs"]
master_df

,Num Farms,Num Estabs,Total Emp Bus
00501,0.0,10.0,10.0
01001,11.0,475.0,486.0
01002,60.0,502.0,562.0
01003,3.0,15.0,18.0
01004,3.0,6.0,9.0
...,...,...,...
99926,1.0,9.0,10.0
99928,1.0,4.0,5.0
99929,2.0,99.0,101.0
99950,3.0,11.0,14.0


In [9]:
race_df = pd.read_csv("Race By ZIP.csv")
race_df["Zip"] = race_df["Geography"].str.slice(-5)
race_df.set_index("Zip", inplace=True)
race_df.head()

,Geography,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Annotation of Margin of Error!!Total:,Annotation of Estimate!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Annotation of Margin of Error!!Total:!!White alone,Annotation of Estimate!!Total:!!White alone,...,Annotation of Margin of Error!!Total:!!Two or more races:,Annotation of Estimate!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Annotation of Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Annotation of Estimate!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Annotation of Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Annotation of Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
Zip,,,,,,,,,,,,,,,,,,,,,
01001,8600000US01001,ZCTA5 01001,16064,941,NaN,NaN,14652,921,NaN,NaN,...,NaN,NaN,50,54,NaN,NaN,287,205,NaN,NaN
01002,8600000US01002,ZCTA5 01002,30099,613,NaN,NaN,22343,921,NaN,NaN,...,NaN,NaN,472,267,NaN,NaN,1750,442,NaN,NaN
01003,8600000US01003,ZCTA5 01003,11588,568,NaN,NaN,7979,554,NaN,NaN,...,NaN,NaN,89,67,NaN,NaN,515,163,NaN,NaN
01005,8600000US01005,ZCTA5 01005,5166,292,NaN,NaN,4877,335,NaN,NaN,...,NaN,NaN,130,126,NaN,NaN,15,33,NaN,NaN
01007,8600000US01007,ZCTA5 01007,15080,21,NaN,NaN,13855,486,NaN,NaN,...,NaN,NaN,48,43,NaN,NaN,366,215,NaN,NaN


In [10]:
race_df["Min Share"] = 1 - (race_df["Estimate!!Total:!!White alone"] / race_df["Estimate!!Total:"])
race_df["Black Share"] = race_df["Estimate!!Total:!!Black or African American alone"] / race_df["Estimate!!Total:"]
race_df

,Geography,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Annotation of Margin of Error!!Total:,Annotation of Estimate!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Annotation of Margin of Error!!Total:!!White alone,Annotation of Estimate!!Total:!!White alone,...,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Annotation of Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Annotation of Estimate!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Annotation of Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Annotation of Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races",Min Share,Black Share
Zip,,,,,,,,,,,,,,,,,,,,,
01001,8600000US01001,ZCTA5 01001,16064,941,NaN,NaN,14652,921,NaN,NaN,...,50,54,NaN,NaN,287,205,NaN,NaN,0.087898,0.028075
01002,8600000US01002,ZCTA5 01002,30099,613,NaN,NaN,22343,921,NaN,NaN,...,472,267,NaN,NaN,1750,442,NaN,NaN,0.257683,0.058108
01003,8600000US01003,ZCTA5 01003,11588,568,NaN,NaN,7979,554,NaN,NaN,...,89,67,NaN,NaN,515,163,NaN,NaN,0.311443,0.044960
01005,8600000US01005,ZCTA5 01005,5166,292,NaN,NaN,4877,335,NaN,NaN,...,130,126,NaN,NaN,15,33,NaN,NaN,0.055943,0.019938
01007,8600000US01007,ZCTA5 01007,15080,21,NaN,NaN,13855,486,NaN,NaN,...,48,43,NaN,NaN,366,215,NaN,NaN,0.081233,0.007427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99923,8600000US99923,ZCTA5 99923,12,17,NaN,NaN,12,17,NaN,NaN,...,0,10,NaN,NaN,0,10,NaN,NaN,0.000000,0.000000
99925,8600000US99925,ZCTA5 99925,990,127,NaN,NaN,479,104,NaN,NaN,...,9,12,NaN,NaN,122,32,NaN,NaN,0.516162,0.008081
99926,8600000US99926,ZCTA5 99926,1582,146,NaN,NaN,174,70,NaN,NaN,...,0,10,NaN,NaN,61,53,NaN,NaN,0.890013,0.000000


In [11]:
master_df = pd.concat([master_df, race_df[["Min Share", "Black Share"]]], axis=1)
values = {"Num Farms":0, "Num Estabs":0, "Total Emp Bus":0}
master_df = master_df.fillna(value=values).sort_index()
master_df

,Num Farms,Num Estabs,Total Emp Bus,Min Share,Black Share
00501,0.0,10.0,10.0,NaN,NaN
01001,11.0,475.0,486.0,0.087898,0.028075
01002,60.0,502.0,562.0,0.257683,0.058108
01003,3.0,15.0,18.0,0.311443,0.044960
01004,3.0,6.0,9.0,NaN,NaN
...,...,...,...,...,...
99927,0.0,0.0,0.0,NaN,NaN
99928,1.0,4.0,5.0,NaN,NaN
99929,2.0,99.0,101.0,0.348346,0.003189
99950,3.0,11.0,14.0,NaN,NaN


In [12]:
master_df.describe()

,Num Farms,Num Estabs,Total Emp Bus,Min Share,Black Share
count,38101.000000,38101.000000,38101.000000,32620.000000,32620.000000
mean,53.600168,213.735650,267.335818,0.180137,0.076940
std,303.917284,446.041546,642.390592,0.213507,0.158188
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,8.000000,22.000000,0.031024,0.000000
50%,20.000000,32.000000,86.000000,0.095238,0.009976
75%,66.000000,221.000000,322.000000,0.252882,0.067460
max,57050.000000,38510.000000,95560.000000,1.000000,1.000000


In [13]:
master_df.drop("99999", inplace=True)
master_df.tail()

,Num Farms,Num Estabs,Total Emp Bus,Min Share,Black Share
99926,1.0,9.0,10.0,0.890013,0.000000
99927,0.0,0.0,0.0,NaN,NaN
99928,1.0,4.0,5.0,NaN,NaN
99929,2.0,99.0,101.0,0.348346,0.003189
99950,3.0,11.0,14.0,NaN,NaN


In [14]:
master_df["Number of Loans_1"] = 0
master_df["Total Loans Amount_1"] = 0
master_df["Total Employees_1"] = 0

master_df["Number of Loans_2"] = 0
master_df["Total Loans Amount_2"] = 0
master_df["Total Employees_2"] = 0

master_df["Number of Loans_3"] = 0
master_df["Total Loans Amount_3"] = 0
master_df["Total Employees_3"] = 0

In [15]:
master_df.to_csv("ZIPdf.csv")

# County Level Data

In [16]:
county_df = pd.read_csv("ZIP-COUNTY-FIPS_2017-06.csv")
county_df = county_df[county_df["STATE"] != "PR"]
county_df["ZIP"] = county_df["ZIP"].astype(str).str.zfill(5)
county_df

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,00501,Suffolk County,NY,36103,H1
190,00802,St. Thomas Island,VI,78030,H4
191,00820,St. Croix Island,VI,78010,H4
192,00830,St. John Island,VI,78020,H4
193,00840,St. Croix Island,VI,78010,H4
...,...,...,...,...,...
52884,99925,Prince of Wales-Hyder Census Area,AK,2198,H5
52885,99926,Prince of Wales-Hyder Census Area,AK,2198,H5
52886,99927,Prince of Wales-Hyder Census Area,AK,2198,H5
52887,99928,Ketchikan Gateway Borough,AK,2130,H1


In [17]:
duplicated_rows = county_df[county_df.duplicated(subset='ZIP', keep=False)]
duplicated_rows

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
316,01002,Franklin County,MA,25011,H4
317,01002,Hampshire County,MA,25015,H4
322,01008,Berkshire County,MA,25003,H4
323,01008,Hampden County,MA,25013,H4
325,01010,Hampden County,MA,25013,H4
...,...,...,...,...,...
52622,99371,Adams County,WA,53001,H1
52623,99371,Franklin County,WA,53021,H1
52626,99403,Asotin County,WA,53003,H1
52627,99403,Garfield County,WA,53023,H1


In [18]:
emp_df = pd.read_csv("Employer Race by County.csv", header = 1)
emp_df = emp_df[["Geographic identifier code", 
                 "Geographic Area Name", 
                 "Sex code", 
                 "Meaning of Sex code", 
                 "Ethnicity code",
                 "Meaning of Ethnicity code", 
                 "Race code",
                 "Meaning of Race code",
                 "Veteran code",
                 "Meaning of Veteran code",
                 "Year",
                 "Number of employer firms"]]
emp_df["FIPS"] = emp_df["Geographic identifier code"].str.slice(-5)
emp_df

,Geographic identifier code,Geographic Area Name,Sex code,Meaning of Sex code,Ethnicity code,Meaning of Ethnicity code,Race code,Meaning of Race code,Veteran code,Meaning of Veteran code,Year,Number of employer firms,FIPS
0,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,0,Total,1,Total,2017,856,01001
1,0500000US01001,"Autauga County, Alabama",2,Female,1,Total,0,Total,1,Total,2017,104,01001
2,0500000US01001,"Autauga County, Alabama",3,Male,1,Total,0,Total,1,Total,2017,S,01001
3,0500000US01001,"Autauga County, Alabama",4,Equally male/female,1,Total,0,Total,1,Total,2017,84,01001
4,0500000US01001,"Autauga County, Alabama",1,Total,20,Hispanic,0,Total,1,Total,2017,S,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51725,0500000US56045,"Weston County, Wyoming",1,Total,1,Total,0,Total,2,Veteran,2017,S,56045
51726,0500000US56045,"Weston County, Wyoming",1,Total,1,Total,0,Total,3,Equally veteran/nonveteran,2017,S,56045
51727,0500000US56045,"Weston County, Wyoming",1,Total,1,Total,0,Total,4,Nonveteran,2017,120,56045
51728,0500000US56045,"Weston County, Wyoming",96,Classifiable,96,Classifiable,96,Classifiable,96,Classifiable,2017,159,56045


In oder to filter rows to just those needed, we filter the values of the other characteristic columns to those only identified as totals.

In [19]:
emp_df_filtered = emp_df[emp_df["Sex code"] == 1]
emp_df_filtered = emp_df_filtered[emp_df_filtered["Ethnicity code"] == 1]
emp_df_filtered = emp_df_filtered[emp_df_filtered["Veteran code"] == 1]
emp_df_filtered

,Geographic identifier code,Geographic Area Name,Sex code,Meaning of Sex code,Ethnicity code,Meaning of Ethnicity code,Race code,Meaning of Race code,Veteran code,Meaning of Veteran code,Year,Number of employer firms,FIPS
0,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,0,Total,1,Total,2017,856,01001
6,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,30,White,1,Total,2017,601,01001
7,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,60,Asian,1,Total,2017,94,01001
8,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,90,Minority,1,Total,2017,107,01001
9,0500000US01001,"Autauga County, Alabama",1,Total,1,Total,91,Equally minority/nonminority,1,Total,2017,S,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51711,0500000US56043,"Washakie County, Wyoming",1,Total,1,Total,91,Equally minority/nonminority,1,Total,2017,S,56043
51712,0500000US56043,"Washakie County, Wyoming",1,Total,1,Total,92,Nonminority,1,Total,2017,203,56043
51718,0500000US56045,"Weston County, Wyoming",1,Total,1,Total,0,Total,1,Total,2017,S,56045
51723,0500000US56045,"Weston County, Wyoming",1,Total,1,Total,30,White,1,Total,2017,159,56045


In [20]:
index = sorted(list(set(emp_df_filtered["FIPS"])))
columns = list(set(emp_df_filtered["Meaning of Race code"]))
owner_df = pd.DataFrame(index = index, columns = columns).fillna(0)
owner_df

,American Indian and Alaska Native,Total,White,Minority,Asian,Black or African American,Nonminority,Native Hawaiian and Other Pacific Islander,Equally minority/nonminority
01001,0,0,0,0,0,0,0,0,0
01003,0,0,0,0,0,0,0,0,0
01005,0,0,0,0,0,0,0,0,0
01007,0,0,0,0,0,0,0,0,0
01009,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
56037,0,0,0,0,0,0,0,0,0
56039,0,0,0,0,0,0,0,0,0
56041,0,0,0,0,0,0,0,0,0
56043,0,0,0,0,0,0,0,0,0


In [21]:
for column in owner_df.columns:
    sub_df = emp_df_filtered[emp_df_filtered["Meaning of Race code"] == column]
    sub_df.set_index("FIPS", inplace=True)
    owner_df[column] = sub_df["Number of employer firms"]
owner_df.replace("S", np.nan, inplace=True)
owner_df["Min Approx"] = owner_df["Total"].astype(np.double) - owner_df["Nonminority"].astype(np.double)
owner_df

,American Indian and Alaska Native,Total,White,Minority,Asian,Black or African American,Nonminority,Native Hawaiian and Other Pacific Islander,Equally minority/nonminority,Min Approx
01001,NaN,856,601,107,94,NaN,588,NaN,NaN,268.0
01003,NaN,4374,3704,263,152,30,3600,NaN,NaN,774.0
01005,NaN,441,340,NaN,NaN,NaN,340,NaN,NaN,101.0
01007,NaN,280,200,NaN,NaN,NaN,178,NaN,NaN,102.0
01009,NaN,586,495,NaN,NaN,NaN,490,NaN,NaN,96.0
...,...,...,...,...,...,...,...,...,...,...
56037,NaN,1026,819,52,NaN,NaN,776,NaN,NaN,250.0
56039,NaN,2023,1793,72,32,NaN,1733,NaN,NaN,290.0
56041,NaN,496,384,NaN,NaN,NaN,371,NaN,NaN,125.0
56043,NaN,259,211,NaN,NaN,NaN,203,NaN,NaN,56.0


In [22]:
missing_values_count = owner_df.isnull().sum()
missing_df = pd.DataFrame(missing_values_count)
missing_df["% missing"] = missing_df[0] / 3140
missing_df["# available"] = 3140 - missing_df[0]
missing_df

,0,% missing,# available
American Indian and Alaska Native,2967,0.944904,173
Total,168,0.053503,2972
White,473,0.150637,2667
Minority,2099,0.668471,1041
Asian,2474,0.787898,666
Black or African American,2770,0.882166,370
Nonminority,488,0.155414,2652
Native Hawaiian and Other Pacific Islander,3103,0.988217,37
Equally minority/nonminority,2827,0.900318,313
Min Approx,517,0.164650,2623


<p><b>Release Date:</b> 2020-05-19</p>

<p>The Census Bureau has reviewed this data product for unauthorized disclosure of confidential information and has approved the disclosure avoidance practices applied (Approval ID: CBDRB-FY20-008).</p>

<p><b>Release Schedule:</b><br />
Data in this file come from estimates of business ownership by sex, ethnicity, race, and veteran status from the 2018 Annual Business Survey (ABS) collection. Data are also obtained from administrative records, the 2017 Economic Census and other economic surveys.</p>

<p><i>Note:</i> The collection year is the year in which the data are collected. A reference year is the year that is referenced in the questions on the survey and in which the statistics are tabulated. For example, the 2018 ABS collection year produces statistics for the 2017 reference year. The "Year" column in the table is the reference year. The ABS has a larger sample size during the benchmark year of 2017. Due to the larger size, more detailed data are shown for reference year 2017.</p>

<p>For more information about ABS planned data product releases, see <a href="https://www.census.gov/programs-surveys/abs/newsroom/updates/tentative-schedule.html">Tentative ABS Schedule</a>.</p>

<p><b>Key Table Information:</b><br />
Includes U.S. firms with paid employees, operating during the reference year with receipts of $1,000 or more, which are classified in the North American Industry Classification System (NAICS), Sectors 11 through 99, except for NAICS 111, 112, 482, 491, 521, 525, 813, 814, and 92 which are not covered. Employer firms with more than one domestic establishment are counted in each geographic area and industry in which they operate, but only once in the U.S. and state totals for all sectors. Employment reflects the number of paid employees during the pay period in the reference year that included March 12.</p>

<p><b>Data Items and Other Identifying Records:</b><br />
Data include estimates on:
<ul><li>Number of employer firms (firms with paid employees)</li>
    <li>Sales and receipts of employer firms (reported in $1,000s of dollars)</li>
    <li>Number of employees (during the March 12 pay period)</li>
    <li>Annual payroll (reported in $1,000s of dollars)</li>
</ul></p>

<p>These data are aggregated by the following demographic classifications of firm for:
<ul><li>All firms
    <ul><li>Classifiable (firms classifiable by sex, ethnicity, race, and veteran status)
    <ul>
	<li>Sex
        <ul><li>Female</li>
            <li>Male</li>
            <li>Equally male/female</li>
        </ul></li>
    <li>Ethnicity
	    <ul><li>Hispanic</li>
            <li>Equally Hispanic/non-Hispanic</li>
            <li>Non-Hispanic</li>
        </ul></li>
    <li>Race
	    <ul><li>White</li>
            <li>Black or African American</li>
            <li>American Indian and Alaska Native</li>
            <li>Asian</li>
            <li>Native Hawaiian and Other Pacific Islander</li>
            <li>Minority (Firms classified as any race and ethnicity combination other than non-Hispanic and White)</li>
            <li>Equally minority/nonminority</li>
            <li>Nonminority (Firms classified as non-Hispanic and White)</li>
        </ul></li>
    <li>Veteran Status (defined as having served in any branch of the U.S. Armed Forces)
	    <ul><li>Veteran</li>
            <li>Equally veteran/nonveteran</li>
            <li>Nonveteran</li>
        </ul></li>
    </ul>
	</li>
        <li>Unclassifiable (firms not classifiable by sex, ethnicity, race, and veteran status)</li>
    </ul></li>
</ul></p>

<p>Moreover, the 2017 reference year statistics include detailed race and ethnicity data tabulated for:
<ul><li>Hispanic subgroups
    <ul><li>Mexican, Mexican American, Chicano</li>
        <li>Puerto Rican</li>
        <li>Cuban</li>
        <li>Other Hispanic, Latino, or Spanish</li>
    </ul></li>
    <li>Asian subgroups
    <ul><li>Asian Indian</li>
        <li>Chinese</li>
        <li>Filipino</li>
        <li>Japanese</li>
        <li>Korean</li>
        <li>Vietnamese</li>
        <li>Other Asian</li>
    </ul></li>
    <li>Native Hawaiian and Other Pacific Islander subgroups
	<ul><li>Native Hawaiian</li>
	<li>Guamanian or Chamorro</li>
        <li>Samoan</li>
        <li>Other Pacific Islander</li>
    </ul></li>
</ul></p>

<p><i>Data Notes:</i>
<ol>
    <li>Business ownership is defined as having 51 percent or more of the stock or equity in the business. Data are provided for businesses owned equally (50% / 50%) by men and women, by Hispanics and non-Hispanics, by minorities and nonminorities, and by veterans and nonveterans. Firms not classifiable by sex, ethnicity, race, and veteran status are counted and tabulated separately.</li>
    <li>The detail may not add to the total or subgroup total because a Hispanic or Latino firm may be of any race, and because a firm could be tabulated in more than one racial group. For example, if a firm responded as both Chinese and Black majority owned, the firm would be included in the detailed Asian and Black estimates but would only be counted once toward the higher level all firms' estimates.</li>
    <li>References such as "Mexican-owned," "Puerto Rican-owned," "Cuban-owned" or "other Hispanic- or Latino-owned" businesses refer only to businesses operating in the 50 states and the District of Columbia that self-identified 51 percent or more of their ownership in 2017 to be by individuals of Mexican, Puerto Rican, Cuban or other Hispanic or Latino origin. The ABS does not distinguish between U.S. residents and nonresidents. Companies owned by foreign governments or owned by other companies, foreign or domestic, are included in the category "Unclassifiable."</li>
</ol></p>

<p><b>Industry and Geography Coverage:</b><br />

The data are shown for the total for all sectors (00) and the 2-, 3-, 4-, 5-, and 6-digit NAICS code levels for:

<ul><li>United States</li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_38">States and the District of Columbia</a></li>
</ul></p>

<p>In addition, the total of all sectors (00) and the 2-digit NAICS code levels are shown for:

<ul><li>Metro Areas, including:
    <ul>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_6">Metropolitan Statistical Areas</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_7">Micropolitan Statistical Areas</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_8">Metropolitan Divisions</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_9">Combined Statistical Areas</a></li>
    </ul></li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage">Counties</a></li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_19">Economic Places</a></li>
</ul></p>

<p>For more information about NAICS, see <a href="https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html">NAICS Codes & Understanding Industry Classification Systems</a>. For information about geographies used by economic programs at the Census Bureau, see <a href="https://www.census.gov/programs-surveys/economic-census/geographies.html">Economic Census: Economic Geographies.</a></p>

<p><b>Footnotes:</b><br />
Footnote 660 - Agriculture, forestry, fishing and hunting (Sector 11): Crop and Animal Production (NAICS 111 and 112) are out of scope.<br />
Footnote 661 - Transportation and warehousing (Sector 48-49): Rail Transportation (NAICS 482) and the Postal Service (NAICS 491) are out of scope.<br />
Footnote 662 - Finance and insurance (Sector 52): Monetary Authorities-Central Banks (NAICS 521) and Funds, Trusts, and Other Financial Vehicles (NAICS 525) are out of scope.<br />
Footnote 663 - Other services, except public administration (Sector 81): Religious, Grantmaking, Civic, Professional, and Similar Organizations (NAICS 813) and Private Households (NAICS 814) are out of scope.</p>

<p><b>FTP Download:</b><br />
Download the entire table at: <a href="https://www2.census.gov/programs-surveys/abs/data/2017/AB1700CSA01.zip">https://www2.census.gov/programs-surveys/abs/data/2017/AB1700CSA01.zip</a>.</p>

<p><b>API Information:</b><br />
Annual Business Survey data are housed in the Census Bureau API. For more information, see <a href="https://api.census.gov/data/2017/abscs.html ">https://api.census.gov/data/2017/abscs.html</a>.</p>

<p><b>Methodology:</b><br />
To maintain confidentiality, the Census Bureau suppresses data to protect the identity of any business or individual. The census results in this file contain sampling and/or nonsampling error. Data users who create their own estimates using data from this file should cite the Census Bureau as the source of the original data only. For information on confidentiality protection, sampling error, nonsampling error, and definitions, see <a href="https://www.census.gov/programs-surveys/abs/technical-documentation/methodology.html">Survey Methodology</a>.</p>

<p><b>Symbols:</b><br />
 <b>S</b> - Estimate does not meet publication standards because of high sampling variability, poor response quality, or other concerns about the estimate quality. Unpublished estimates derived from this table by subtraction are subject to these same limitations and should not be attributed to the U.S. Census Bureau. For a description of publication standards and the total quantity response rate, see link to program methodology <a href="https://www.census.gov/programs-surveys/abs/technical-documentation/methodology.html">page</a>.<br />
 <b>N</b> - Not available or not comparable<br />
 <b>X</b> - Not applicable<br />
For a complete list of all economic programs symbols, see the <a href="https://www.census.gov/programs-surveys/economic-census/technical-documentation/data-dictionary.html">Symbols Glossary</a>.</p>

<p><b>Source:</b><br />
U.S. Census Bureau, Annual Business Survey (ABS)<br />
For more information about the survey, please visit <a href="https://www.census.gov/programs-surveys/abs.html">https://www.census.gov/programs-surveys/abs.html</a>.</p>

<p><b>Contact Information:</b><br />
To contact the Annual Business Survey staff:
<ul><li>Email general, nonsecure, and unencrypted messages to <a href="mailto:adep.annual.business.survey@census.gov">adep.annual.business.survey@census.gov</a>.</li>
    <li>Call 301.763.3316 between 7 a.m. and 5 p.m. (EST), Monday through Friday.</li>
</ul>
</p>
